In [1]:
import os
from bs4 import BeautifulSoup
import time

In [2]:
SEASONS = list(range(2022,2025))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
import requests 
def get_html_by_selector(url, selector):
    try:
        # Effectuer une requête HTTP GET à l'URL
        response = requests.get(url)
        response.raise_for_status()  # Lèvera une exception en cas de problème avec la requête

        # Vérifier le type de contenu pour s'assurer que c'est du texte/html
        content_type = response.headers['Content-Type'].lower()
        if 'text/html' not in content_type:
            raise ValueError(f"Le contenu n'est pas HTML: {content_type}")

        # Utiliser BeautifulSoup pour analyser le contenu HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        # Imprimer le titre de la page
        title = soup.title.text.strip()
        print(f"Titre de la page: {title}")

        # Récupérer le contenu HTML du sélecteur spécifié
        selected_html = soup.select_one(selector)
        if selected_html:
            return str(selected_html)

        raise ValueError(f"Sélecteur '{selector}' non trouvé sur la page {url}")

    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête vers {url}: {e}")
        return None
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'analyse HTML de {url}: {e}")
        return None

In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = get_html_by_selector(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = get_html_by_selector(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [5]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
standings_files = os.listdir(STANDINGS_DIR)


In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]

    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)

        await scrape_game(filepath)